In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import numpy as np
device = "cuda"
# Transform to normalize the data
transform = transforms.Compose([transforms.ToTensor()])
# Load the training and test datasets
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=256, shuffle=True)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = DataLoader(testset, batch_size=256, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [2]:
class Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 3)
        self.pool = nn.AvgPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 3)
        self.conv3 = nn.Conv2d(16, 32, 3)
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=128,
            nhead=8,
            dim_feedforward=512,
            dropout=.35,
            batch_first=True
        )
        self.te = nn.TransformerEncoder(encoder_layer, num_layers=4)
        self.fc = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = torch.flatten(x, 1)
        x = self.te(x)
        x = self.fc(x)
        return x


class DenoiseModel(nn.Module):
    def __init__(self):
        super(DenoiseModel, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(4, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 3, kernel_size=3, padding=1),
        )

    def forward(self, x, t):
        t = torch.full((x.size(0), 1, x.size(2), x.size(3)), t, device=x.device, dtype=x.dtype)
        x = torch.cat([x, t], dim=1)
        return self.main(x)

In [ ]:
classifier = Classifier().to(device)
opt_cls = optim.AdamW(classifier.parameters(), lr=5e-5)
crt_cls = nn.CrossEntropyLoss()
for epoch in range(1000):
    running_loss = 0.0
    train_size = 0
    for i, data in enumerate(trainloader):
        train_size += len(data)
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        opt_cls.zero_grad()
        outputs = classifier(inputs)
        loss = crt_cls(outputs, labels)
        loss.backward()
        opt_cls.step()
        running_loss += loss.item()
    print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss /train_size :.3f}')
    running_loss = 0.0
    
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(device), labels.to(device)
            outputs = classifier(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

print('Finished Training')

In [17]:
PATH = './cifar_cls.pth'
torch.save(classifier.state_dict(), PATH)

In [66]:
model = DenoiseModel().to(device)
optimizer = optim.AdamW(model.parameters(), 5e-4)
criterion = nn.MSELoss()
num_epochs = 1000
start_t = 0.3 
end_t = 0.01 
t_decrement = (start_t - end_t) / (num_epochs - 1)
t = start_t

In [67]:

for epoch in range(num_epochs):
    running_loss = 0.0
    train_size = 0
    t = max(end_t, t - t_decrement)
    for data in trainloader:
        train_size += len(data)
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        noise = torch.randn_like(images) 
        noisy_images = images + noise * t
        optimizer.zero_grad()
        pred_noise = model(noisy_images, t)
        loss = criterion(pred_noise, noise)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'[{epoch + 1}, {i + 1:5d}] loss: {1000 * running_loss / train_size :.3f}')


[1,     1] loss: 170.143
[2,     1] loss: 62.994
[3,     1] loss: 59.190
[4,     1] loss: 56.749
[5,     1] loss: 54.815
[6,     1] loss: 53.384
[7,     1] loss: 51.840
[8,     1] loss: 50.356
[9,     1] loss: 48.889
[10,     1] loss: 47.344
[11,     1] loss: 45.848
[12,     1] loss: 44.267
[13,     1] loss: 43.005
[14,     1] loss: 41.998
[15,     1] loss: 41.305
[16,     1] loss: 40.720
[17,     1] loss: 40.367
[18,     1] loss: 40.088
[19,     1] loss: 39.854
[20,     1] loss: 39.679
[21,     1] loss: 39.580
[22,     1] loss: 39.431
[23,     1] loss: 39.316
[24,     1] loss: 38.999
[25,     1] loss: 38.568


In [3]:
# classifier = None
# denoiser = None

num_epochs = 1000
start_t = 0.3 
end_t = 0.01 
t_decrement = (start_t - end_t) / (num_epochs - 1)
t = start_t

In [5]:
aug_cls = Classifier().to(device)

opt_aug = optim.AdamW(aug_cls.parameters(), lr=5e-5)
crt_aug = nn.CrossEntropyLoss()
for epoch in range(num_epochs):
    running_loss = 0.0
    train_size = 0
    t = max(end_t, t - t_decrement)
    for i, data in enumerate(trainloader):
        train_size += len(data)
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        noise = torch.randn_like(images) 
        noisy_images = images + noise * t
        opt_aug.zero_grad()
        outputs = aug_cls(images)
        loss = crt_aug(outputs, labels)
        loss.backward()
        opt_aug.step()
        running_loss += loss.item()
    print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss /train_size :.3f}')
    running_loss = 0.0
    
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(device), labels.to(device)
            outputs = aug_cls(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')
print('Finished Training')

NameError: name 'inputs' is not defined